In [1]:
# used for statistical processes, i.e scaling the dataset

from lstm_models import LSTM
from gpu_dataloader import GPUDataset
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# plotting the data
import matplotlib.pyplot as plt
# used for the dataframes
import pandas as pd

import numpy as np
%matplotlib inline


In [2]:
import math

def get_rmse(actual_values, predicted_values) -> float:
    '''returns the root mean squared error'''
    return math.sqrt(mean_squared_error(actual_values, predicted_values))

def get_mape(actual_values, predicted_values):
    '''returns the mean absolue percentage error'''
    return np.mean(np.abs(actual_values - predicted_values) / np.abs(actual_values) * 100)

def get_mae(actual_values, predicted_values) -> float:
    '''returns the mean absolute error'''
    return mean_absolute_error(actual_values, predicted_values)

In [3]:
batch_size: int = 500
dataset = GPUDataset(small_df=True, batch_size=batch_size)

In [4]:
dataset.X.shape

torch.Size([5000, 1, 19])

In [5]:
num_epochs: int = 50
learning_rate: float = 0.02

# number of features
input_size: int = dataset.X.shape[2]
# number of features in hidden state
hidden_size: int = dataset.X.shape[2] * 32
# number of stacked lstm layers
num_layers: int = 1
# number of output classes
num_classes: int = dataset.y.shape[1]

In [6]:
import wandb
wandb.init(project='Hardware Utilization Prediction')

wandb.config.num_epochs = num_epochs
wandb.config.learning_rate = learning_rate
wandb.config.input_size = input_size
wandb.config.hidden_size = hidden_size
wandb.config.num_layers = num_layers
wandb.config.num_classes = num_classes

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: my-god-its-full-of-stars. Use `wandb login --relogin` to force relogin


In [7]:
lstm = LSTM(num_classes, input_size, hidden_size, num_layers, dataset.X.shape[1])
lstm.train()

LSTM(
  (lstm): LSTM(19, 608, batch_first=True)
  (fc_1): Linear(in_features=608, out_features=256, bias=True)
  (fc_2): Linear(in_features=256, out_features=128, bias=True)
  (fc_3): Linear(in_features=128, out_features=3, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
)

In [8]:
# mean squared error for regression
criterion = nn.MSELoss()
# optimizer function
optimizer = torch.optim.AdamW(lstm.parameters(), lr=learning_rate)

In [9]:
LOSS: str = 'loss'
RMSE_TRAINING: str = 'root mean squared error (training)'
MAE_TRAINING: str = 'mean absolute error (training)'

wandb.define_metric(LOSS, summary='min')
wandb.define_metric(RMSE_TRAINING, summary='min')
wandb.define_metric(MAE_TRAINING, summary='min')

In [10]:
import random

for epoch in range(num_epochs):
    print(f'Epoch: {epoch}')

    batch_section = [idx for idx in range(len(dataset) // batch_size)]

    while len(batch_section) > 0:
        choice = random.choice(batch_section)
        sampler = SubsetRandomSampler(
            list(range(choice*batch_size, (choice+1)*batch_size)))
        train_loader = DataLoader(
            dataset, batch_size=batch_size, shuffle=False, num_workers=5, sampler=sampler)

        for _, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()

            outputs = lstm.forward(inputs)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # remove batch section after training on it
        batch_section.remove(choice)
        

    # logging to wandb
    o = outputs.detach()
    rmse = get_rmse(o, labels)
    mae = get_mae(o, labels)
    log_dict: dict = {
        LOSS: loss.item(),
        RMSE_TRAINING: rmse,
        MAE_TRAINING: mae,
    }
    wandb.log(log_dict)

    if epoch % 5 == 0:
        print(
            f'Epoch: {epoch}, loss: {loss.item():2f}, rmse: {rmse:2f}, mae: {mae:2f}')


Epoch: 1
Epoch: 1, loss: 0.015989, rmse: 0.126447, mae: 0.080706
Epoch: 2
Epoch: 3


In [ ]:
import time

current_time = time.ctime()
current_time

## Save the Model to Disk

In [ ]:
# torch.save(lstm.state_dict(), f'models/epochs-{num_epochs}-{current_time}')

In [ ]:
lstm.eval()

# forward pass
prediction = lstm(dataset.X)
prediction = prediction.data.numpy()
prediction = dataset.standard_scaler.fit_transform(prediction)

actual_data = dataset.y.data.numpy()
actual_data = dataset.minmax_scaler.fit_transform(actual_data)

# reverse transformation
prediction = dataset.standard_scaler.inverse_transform(prediction)
actual_data = dataset.minmax_scaler.inverse_transform(actual_data)

label_columns = dataset.get_default_label_columns()
# create dataframes
prediction_df = pd.DataFrame(prediction, columns=label_columns)
actual_data_df = pd.DataFrame(actual_data, columns=label_columns)


## Calculate Root Mean Squared Error

Calculating the RMSE for the overall prediction of the (training) dataset.

In [ ]:
rmse_key: str = 'Root Mean Squared Error (Overall - Training)'
rmse_result = get_rmse(actual_data_df[:], prediction_df[:])
print(f'Test Score: {rmse_result:.2f} RMSE')
wandb.summary[rmse_key] = rmse_result

## Calculate Mean Absolute Error

Calcutlate the MAE for the overall prediction of the (training) dataset.

In [ ]:
mae_key: str = 'Mean Absolute Error (Overall - Training)'
mae_result = mean_absolute_error(actual_data_df[:], prediction_df[:])
print(f'Test Score: {mae_result} MAE')
wandb.summary[mae_key] = mae_result

In [ ]:
get_rmse(actual_data_df[actual_data_df.columns[1]], actual_data_df[actual_data_df.columns[4]])

In [ ]:
def plot_column(actual_values=actual_data_df, predicted_values=prediction_df, column_number: int = 0, rmse_threshold: float = 0.30):

    if len(label_columns) <= column_number:
        print('Out of Prediction Bounds')
        return

    plt.figure(figsize=(25, 15))  # plotting

    column = label_columns[column_number]
    pred_column = f'pred_{column}'

    rmse = get_rmse(actual_values[column], predicted_values[column])
    mae = mean_absolute_error(actual_values[column], predicted_values[column])

    predicted_color = 'green' if rmse < rmse_threshold else 'orange'

    plt.plot(actual_values[column], label=column, color='black')  # actual plot
    plt.plot(predicted_values[column], label='pred_' +
             column, color=predicted_color)  # predicted plot

    plt.title('Time-Series Prediction')
    plt.plot([], [], ' ', label=f'RMSE: {rmse}')
    plt.plot([], [], ' ', label=f'MAE: {mae}')
    plt.legend()
    
    wandb.log({pred_column: wandb.Image(plt)})
    wandb.summary[f'Root Mean Squared Error ({column})'] = rmse
    wandb.summary[f'Mean Absolute Error ({column})'] = mae
    
    plt.show()


## See Predictions on Training Dataset

In [ ]:
plot_column(column_number=0)

In [ ]:
plot_column(column_number=1)

In [ ]:
plot_column(column_number=2)

In [ ]:
plot_column(column_number=3)

In [ ]:
plot_column(column_number=4)

In [ ]:
plot_column(column_number=5)

In [ ]:
plot_column(column_number=6)